In [1]:
# https://colab.research.google.com/drive/1oaHpvWKjFexrZl9zXoSivNO_VsmTneNQ#scrollTo=ZN9I0nIBZ_vW 코랩사본링크

import numpy as np
import torch
# from pkg_resource import packaging
# print(torch.__version__)
import clip

#사용가능한 모델
# clip.available_model()
# ['RN50',
#  'RN101',
#  'RN50x4',
#  'RN50x16',
#  'RN50x64',
#  'ViT-B/32',
#  'ViT-B/16',
#  'ViT-L/14',
#  'ViT-L/14@336px']


#모델 호출
model, preprocess = clip.load("ViT-B/16")
#preprocess 확인
print('preprocess : ', preprocess)

#text Preprocessing
text_tokens = clip.tokenize("우주에 고양이가 있다.")
#clip()의 tokenize 안에는 simpletokenizer가 있음
#출력결과 2차원 탠서 
print(text_tokens)


#Buildiing features
# image_input = torch.tensor(np.stack(images)).cuda()

c:\ProgramData\Anaconda3\envs\PJ3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


preprocess :  Compose(
    Resize(size=224, interpolation=PIL.Image.BICUBIC)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x0000016929BB4550>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)
tensor([[49406, 38415, 45161, 13252,   494, 36216,   254, 31625,   239, 12286,
         21122,   478, 20849,   230, 13094,   353,   269, 49407,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0]])


In [2]:
text_tokens.shape

torch.Size([1, 77])

In [13]:
#이미지 넘파이로 다루기 : https://suhwanc.tistory.com/94

In [15]:
from PIL import Image
 
image = Image.open("우주고양이.jpg")
image.show()

In [16]:
image = np.asarray(image)

In [17]:
image.shape

(500, 800, 3)

In [8]:
texts = "우주에 고양이"
image = preprocess(Image.open("우주고양이.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["우주", "고양이", "강아지"]).to(device)

In [12]:
image_input = torch.tensor(np.stack(image)).cuda()
text_tokens = clip.tokenize(["This is " + desc for desc in texts]).cuda()

In [18]:
#차원이 맞지 않음
with torch.no_grad():
    image_features = model.encode_image(image_input).float()
    text_features = model.encode_text(text_tokens).float()

RuntimeError: Expected 4-dimensional input for 4-dimensional weight [768, 3, 16, 16], but got 1-dimensional input of size [3] instead

# 다시

In [19]:
# https://github.com/openai/CLIP
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("우주고양이.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["우주", "고양이", "동물"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # prints: [[0.9927937  0.00421068 0.00299572]]

Label probs: [[0.03842 0.65    0.3118 ]]


In [ ]:
#clip-retrieval